In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e1a2d7372b83ef3aad7f69a01522e92fb58924b6e02f3ce98389b851eabb3a15
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [3]:
sc = SparkContext(conf=SparkConf().setMaster('local[8]'))
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/08 13:40:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/08 13:40:48 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [4]:
# df = pd.read_csv('../input/yago-full-clean/yago_full_clean.txt',sep = ' ', header=None) 
df1 = spark.read.csv("../input/small-dataset/small_input.txt", sep = ' ', header=None)

In [5]:
print((df1.count(), len(df1.columns)))

(300000, 3)


In [6]:
df1.show()

+--------------------+------------+--------------------+
|                 _c0|         _c1|                 _c2|
+--------------------+------------+--------------------+
|<Jesús_Rivera_Sán...|<isLeaderOf>|<Pueblo_of_Naranj...|
|      <Elizabeth_II>|<isLeaderOf>|<Royal_Numismatic...|
|  <Richard_Stallman>|<isLeaderOf>|<Free_Software_Fo...|
|    <Keith_Peterson>|<isLeaderOf>|     <Cambridge_Bay>|
|<William_H._Sewar...|<isLeaderOf>|<9th_New_York_Hea...|
|          <Andranik>|<isLeaderOf>|   <Armenian_fedayi>|
|<Ramasamy_Palanis...|<isLeaderOf>|<Democratic_Actio...|
|        <Matt_Bevin>|<isLeaderOf>|<Kentucky_Air_Nat...|
|       <Leonard_Leo>|<isLeaderOf>|<Federalist_Society>|
|<James_Vincent_Cl...|<isLeaderOf>|<Bennington,_New_...|
|      <Amjad_Bashir>|<isLeaderOf>|  <Khushab_District>|
|<Tushar_Amarsinh_...|<isLeaderOf>|     <Mota,_Gujarat>|
|  <Yiannis_Boutaris>|<isLeaderOf>|      <Thessaloniki>|
|<Margaret_Chew_Ba...|<isLeaderOf>|  <American_INSIGHT>|
|     <Oscar_Braynon>|<isLeader

In [7]:
# df = df1.sample(False, 0.05)

In [8]:
# df = df1.limit(1000)
df = df1

In [9]:
print((df.count(), len(df.columns)))

(300000, 3)


In [10]:
df.show()

+--------------------+------------+--------------------+
|                 _c0|         _c1|                 _c2|
+--------------------+------------+--------------------+
|<Jesús_Rivera_Sán...|<isLeaderOf>|<Pueblo_of_Naranj...|
|      <Elizabeth_II>|<isLeaderOf>|<Royal_Numismatic...|
|  <Richard_Stallman>|<isLeaderOf>|<Free_Software_Fo...|
|    <Keith_Peterson>|<isLeaderOf>|     <Cambridge_Bay>|
|<William_H._Sewar...|<isLeaderOf>|<9th_New_York_Hea...|
|          <Andranik>|<isLeaderOf>|   <Armenian_fedayi>|
|<Ramasamy_Palanis...|<isLeaderOf>|<Democratic_Actio...|
|        <Matt_Bevin>|<isLeaderOf>|<Kentucky_Air_Nat...|
|       <Leonard_Leo>|<isLeaderOf>|<Federalist_Society>|
|<James_Vincent_Cl...|<isLeaderOf>|<Bennington,_New_...|
|      <Amjad_Bashir>|<isLeaderOf>|  <Khushab_District>|
|<Tushar_Amarsinh_...|<isLeaderOf>|     <Mota,_Gujarat>|
|  <Yiannis_Boutaris>|<isLeaderOf>|      <Thessaloniki>|
|<Margaret_Chew_Ba...|<isLeaderOf>|  <American_INSIGHT>|
|     <Oscar_Braynon>|<isLeader

In [11]:
df = df.withColumnRenamed("_c0","entity1")
df = df.withColumnRenamed("_c1","relation")
df = df.withColumnRenamed("_c2","entity2")

In [12]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when

In [13]:
df = df.withColumn('entity1', 
    when(df.entity1.startswith('<'),regexp_replace(df.entity1,'<',''))\
   .otherwise(df.entity1)) \
   .toDF("entity1","relation","entity2")

df = df.withColumn('entity1', 
    when(df.entity1.endswith('>'),regexp_replace(df.entity1,'>',''))\
   .otherwise(df.entity1)) \
   .toDF("entity1","relation","entity2")

In [14]:
df = df.withColumn('relation', 
    when(df.relation.startswith('<'),regexp_replace(df.relation,'<',''))\
   .otherwise(df.relation)) \
   .toDF("entity1","relation","entity2")

df = df.withColumn('relation', 
    when(df.relation.endswith('>'),regexp_replace(df.relation,'>',''))\
   .otherwise(df.relation)) \
   .toDF("entity1","relation","entity2")

In [15]:
df = df.withColumn('entity2', 
    when(df.entity2.startswith('<'),regexp_replace(df.entity2,'<',''))\
   .otherwise(df.entity2)) \
   .toDF("entity1","relation","entity2")

df = df.withColumn('entity2', 
    when(df.entity2.endswith('>'),regexp_replace(df.entity2,'>',''))\
   .otherwise(df.entity2)) \
   .toDF("entity1","relation","entity2")

In [16]:
df.show()

+--------------------+----------+--------------------+
|             entity1|  relation|             entity2|
+--------------------+----------+--------------------+
|Jesús_Rivera_Sánchez|isLeaderOf| Pueblo_of_Naranjito|
|        Elizabeth_II|isLeaderOf|Royal_Numismatic_...|
|    Richard_Stallman|isLeaderOf|Free_Software_Fou...|
|      Keith_Peterson|isLeaderOf|       Cambridge_Bay|
|William_H._Seward...|isLeaderOf|9th_New_York_Heav...|
|            Andranik|isLeaderOf|     Armenian_fedayi|
| Ramasamy_Palanisamy|isLeaderOf|Democratic_Action...|
|          Matt_Bevin|isLeaderOf|Kentucky_Air_Nati...|
|         Leonard_Leo|isLeaderOf|  Federalist_Society|
|James_Vincent_Cleary|isLeaderOf|Bennington,_New_H...|
|        Amjad_Bashir|isLeaderOf|    Khushab_District|
|Tushar_Amarsinh_C...|isLeaderOf|       Mota,_Gujarat|
|    Yiannis_Boutaris|isLeaderOf|        Thessaloniki|
|Margaret_Chew_Bar...|isLeaderOf|    American_INSIGHT|
|       Oscar_Braynon|isLeaderOf|          Allapattah|
|Margrethe

In [17]:
unique_words = set()

for row in df.collect():
    unique_words.add(row["entity1"])
    unique_words.add(row["relation"])
    unique_words.add(row["entity2"])

In [18]:
len(unique_words)

306549

In [19]:
values_int = list(range(1, len(unique_words)+1))
values_string = [str(x) for x in values_int]
values = set(values_string)

In [20]:
type(values_string)

list

In [21]:
from collections import Counter

In [22]:
items = Counter(values).keys()
print(len(items))

306549


In [23]:
unique_dict = dict(zip(unique_words, values))

In [24]:
len(unique_dict)

306549

In [25]:
df_encoded = df.rdd.map(lambda x: (unique_dict[x.entity1], unique_dict[x.relation], unique_dict[x.entity2])).toDF(["entity1","relation","entity2"])

22/08/08 13:41:08 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [26]:
df_encoded.show()

+-------+--------+-------+
|entity1|relation|entity2|
+-------+--------+-------+
| 258009|   71565|  28639|
| 285059|   71565|  81407|
|  70958|   71565| 254198|
| 239247|   71565|  96294|
|  12396|   71565| 232687|
| 168741|   71565|  62325|
| 256457|   71565| 282033|
| 229542|   71565| 255381|
| 263751|   71565|  65730|
| 300961|   71565| 306117|
|  31008|   71565| 255717|
| 219579|   71565| 244583|
| 227608|   71565| 292971|
| 120135|   71565| 194506|
|  93579|   71565|  79887|
|  59283|   71565| 247426|
| 112030|   71565| 193361|
|  56397|   71565|  59831|
|  93099|   71565| 212937|
| 292749|   71565|  83356|
+-------+--------+-------+
only showing top 20 rows



In [27]:
t1 = df_encoded
t2 = df_encoded

In [28]:
t2.printSchema()

root
 |-- entity1: string (nullable = true)
 |-- relation: string (nullable = true)
 |-- entity2: string (nullable = true)



In [29]:
t_2 = t2.selectExpr("entity1 as entity3", "relation as relation2", "entity2 as entity4")

In [30]:
t_2.show()

+-------+---------+-------+
|entity3|relation2|entity4|
+-------+---------+-------+
| 258009|    71565|  28639|
| 285059|    71565|  81407|
|  70958|    71565| 254198|
| 239247|    71565|  96294|
|  12396|    71565| 232687|
| 168741|    71565|  62325|
| 256457|    71565| 282033|
| 229542|    71565| 255381|
| 263751|    71565|  65730|
| 300961|    71565| 306117|
|  31008|    71565| 255717|
| 219579|    71565| 244583|
| 227608|    71565| 292971|
| 120135|    71565| 194506|
|  93579|    71565|  79887|
|  59283|    71565| 247426|
| 112030|    71565| 193361|
|  56397|    71565|  59831|
|  93099|    71565| 212937|
| 292749|    71565|  83356|
+-------+---------+-------+
only showing top 20 rows



In [31]:
df_join = t1.join(t_2, t1['entity2']==t_2['entity3'])

In [32]:
df_join.show()

+-------+--------+-------+-------+---------+-------+
|entity1|relation|entity2|entity3|relation2|entity4|
+-------+--------+-------+-------+---------+-------+
|  14270|   71565| 102453| 102453|    83362|  38393|
|  67388|   71565| 102453| 102453|    83362|  38393|
|  30098|   83362| 103024| 103024|    83362| 279731|
| 304869|   83362| 103024| 103024|    83362| 279731|
| 178583|   83362| 103099| 103099|    83362| 238678|
| 142680|   83362| 107697| 107697|    83362| 246113|
| 142680|   83362| 107697| 107697|    83362| 106589|
| 142680|   83362| 107697| 107697|    83362|  30383|
| 142680|   83362| 107697| 107697|    83362|  80690|
| 142680|   83362| 107697| 107697|    83362| 145055|
|  90778|   83362| 107697| 107697|    83362| 246113|
|  90778|   83362| 107697| 107697|    83362| 106589|
|  90778|   83362| 107697| 107697|    83362|  30383|
|  90778|   83362| 107697| 107697|    83362|  80690|
|  90778|   83362| 107697| 107697|    83362| 145055|
| 246115|   83362| 108893| 108893|    83362|  

In [33]:
df_join = df_join.drop("entity3")

In [34]:
df_join.show()

+-------+--------+-------+---------+-------+
|entity1|relation|entity2|relation2|entity4|
+-------+--------+-------+---------+-------+
|  14270|   71565| 102453|    83362|  38393|
|  67388|   71565| 102453|    83362|  38393|
|  30098|   83362| 103024|    83362| 279731|
| 304869|   83362| 103024|    83362| 279731|
| 178583|   83362| 103099|    83362| 238678|
| 142680|   83362| 107697|    83362| 246113|
| 142680|   83362| 107697|    83362| 106589|
| 142680|   83362| 107697|    83362|  30383|
| 142680|   83362| 107697|    83362|  80690|
| 142680|   83362| 107697|    83362| 145055|
|  90778|   83362| 107697|    83362| 246113|
|  90778|   83362| 107697|    83362| 106589|
|  90778|   83362| 107697|    83362|  30383|
|  90778|   83362| 107697|    83362|  80690|
|  90778|   83362| 107697|    83362| 145055|
| 246115|   83362| 108893|    83362|  96256|
| 246115|   83362| 108893|    83362| 120564|
| 246115|   83362| 108893|    83362|  67321|
| 246115|   83362| 108893|    83362|  21796|
| 246115| 

In [35]:
# columns = list(df_join1)
# print(columns)
df_join.dtypes

[('entity1', 'string'),
 ('relation', 'string'),
 ('entity2', 'string'),
 ('relation2', 'string'),
 ('entity4', 'string')]

In [36]:
print((df_join.count(), len(df_join.columns)))

(3296930, 5)


In [37]:
df_join.show()

+-------+--------+-------+---------+-------+
|entity1|relation|entity2|relation2|entity4|
+-------+--------+-------+---------+-------+
|  14270|   71565| 102453|    83362|  38393|
|  67388|   71565| 102453|    83362|  38393|
|  30098|   83362| 103024|    83362| 279731|
| 304869|   83362| 103024|    83362| 279731|
| 178583|   83362| 103099|    83362| 238678|
| 142680|   83362| 107697|    83362| 246113|
| 142680|   83362| 107697|    83362| 106589|
| 142680|   83362| 107697|    83362|  30383|
| 142680|   83362| 107697|    83362|  80690|
| 142680|   83362| 107697|    83362| 145055|
|  90778|   83362| 107697|    83362| 246113|
|  90778|   83362| 107697|    83362| 106589|
|  90778|   83362| 107697|    83362|  30383|
|  90778|   83362| 107697|    83362|  80690|
|  90778|   83362| 107697|    83362| 145055|
| 246115|   83362| 108893|    83362|  96256|
| 246115|   83362| 108893|    83362| 120564|
| 246115|   83362| 108893|    83362|  67321|
| 246115|   83362| 108893|    83362|  21796|
| 246115| 

In [38]:
import pyspark.sql.functions as f

columns = [f.col("entity1"), f.col("relation"), f.col("entity2"), f.col("relation2"), f.col("entity4")] 

output = df_join.withColumn("tokens", f.array(columns)).select("tokens")

In [39]:
output.show(truncate=False)

+--------------------------------------+
|tokens                                |
+--------------------------------------+
|[14270, 71565, 102453, 83362, 38393]  |
|[67388, 71565, 102453, 83362, 38393]  |
|[30098, 83362, 103024, 83362, 279731] |
|[304869, 83362, 103024, 83362, 279731]|
|[178583, 83362, 103099, 83362, 238678]|
|[142680, 83362, 107697, 83362, 246113]|
|[142680, 83362, 107697, 83362, 106589]|
|[142680, 83362, 107697, 83362, 30383] |
|[142680, 83362, 107697, 83362, 80690] |
|[142680, 83362, 107697, 83362, 145055]|
|[90778, 83362, 107697, 83362, 246113] |
|[90778, 83362, 107697, 83362, 106589] |
|[90778, 83362, 107697, 83362, 30383]  |
|[90778, 83362, 107697, 83362, 80690]  |
|[90778, 83362, 107697, 83362, 145055] |
|[246115, 83362, 108893, 83362, 96256] |
|[246115, 83362, 108893, 83362, 120564]|
|[246115, 83362, 108893, 83362, 67321] |
|[246115, 83362, 108893, 83362, 21796] |
|[246115, 83362, 108893, 83362, 261613]|
+--------------------------------------+
only showing top

In [40]:
print((output.count(), len(output.columns)))

(3296930, 1)


In [41]:
tokens = output.select("tokens")

In [42]:
tokens.show()

+--------------------+
|              tokens|
+--------------------+
|[14270, 71565, 10...|
|[67388, 71565, 10...|
|[30098, 83362, 10...|
|[304869, 83362, 1...|
|[178583, 83362, 1...|
|[142680, 83362, 1...|
|[142680, 83362, 1...|
|[142680, 83362, 1...|
|[142680, 83362, 1...|
|[142680, 83362, 1...|
|[90778, 83362, 10...|
|[90778, 83362, 10...|
|[90778, 83362, 10...|
|[90778, 83362, 10...|
|[90778, 83362, 10...|
|[246115, 83362, 1...|
|[246115, 83362, 1...|
|[246115, 83362, 1...|
|[246115, 83362, 1...|
|[246115, 83362, 1...|
+--------------------+
only showing top 20 rows



In [43]:
type(tokens)

pyspark.sql.dataframe.DataFrame

In [44]:
inp_list = output.select('tokens').rdd.flatMap(lambda x: x).collect()

22/08/08 13:42:13 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/08/08 13:42:13 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/08/08 13:42:13 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/08/08 13:42:13 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/08/08 13:42:13 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the f

22/08/08 13:42:21 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/08/08 13:42:21 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/08/08 13:42:21 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/08/08 13:42:21 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/08/08 13:42:21 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the f

In [45]:
len(inp_list)

3296930

In [46]:
# output.select("tokens").collect()[0]

In [47]:
# input_list = []
# for rows in output.select("tokens").collect():
#     input_list.append(rows[0])

In [48]:
import gensim

In [49]:
model = gensim.models.Word2Vec(min_count=1,
                               vector_size= 850,
                               workers=4,
                               window =4,
                               sg = 1)

In [50]:
model.build_vocab(inp_list, progress_per=10000)

In [51]:
len(model.wv.key_to_index)

195756

In [52]:
#model.wv.key_to_index

In [53]:
model.train(inp_list, total_examples=model.corpus_count, epochs=model.epochs)

(40140784, 82423250)

In [54]:
val_list = list(unique_dict.values())
key_list = list(unique_dict.keys())

In [55]:
print(len(unique_dict))
len(model.wv.key_to_index)

306549


195756

In [56]:
type(val_list[0])

str

In [57]:
def word_to_w2v_generated(word):
#     print(unique_dict[word])
    return model.wv.key_to_index[unique_dict[word]]

In [58]:
# print("Word:", key_list[val_list.index(913)])
# print("Manually_generated:", 913)
# print("Word_to_vec genetated:", model.wv.key_to_index[913])

In [59]:
word_to_w2v_generated("Godavari_Dam")

246

In [60]:
# model.wv[word_to_w2v_generated("Godavari_Dam")]

In [61]:
import numpy as np
from numpy.linalg import norm

In [62]:
# other_embedding_of_GodavariDam =  [ 1.3730227 , -0.84648895, -1.1720586 ,  0.44773513, -0.8799845 ,
#        -0.6401834 ,  0.27228343,  0.6808291 ,  0.07503165, -0.88640815,
#         1.0291374 ,  2.160502  , -1.9196916 , -0.6732352 ,  0.9821364 ,
#         0.8785317 ,  0.39054427, -0.2155897 , -0.12747395, -0.21941929,
#         0.9536586 ,  1.5859957 ,  1.4195433 , -0.8667842 ,  0.06520381,
#         0.2562395 ,  0.87870806,  0.996617  , -0.254336  , -0.9206493 ,
#        -0.35556167, -0.69568384,  0.98694324,  0.23072457,  1.0978839 ,
#        -0.98354536,  1.2168041 , -0.7158572 , -1.6860644 , -0.10616143,
#        -1.0365301 , -0.17086948, -1.2599028 , -0.90408444,  1.4062765 ,
#         0.5066581 , -0.7715973 , -0.8436604 ,  0.15885781,  1.3730444 ]

# other_embedding_of_GodavariDam = np.array(other_embedding_of_GodavariDam)

In [63]:
# np.dot(other_embedding_of_GodavariDam,model.wv[word_to_w2v_generated("Godavari_Dam")])/(norm(other_embedding_of_GodavariDam)*norm(model.wv[word_to_w2v_generated("Godavari_Dam")]))

In [64]:
# other_embedding_of_UnitedStates =  [-0.25793448, -0.2010411 , -0.9153717 ,  0.3798568 ,  0.3105798 ,
#         0.7572765 ,  0.5634417 ,  0.47487262,  1.1290884 , -0.6045809 ,
#        -0.7380967 ,  0.32126337, -0.6979069 , -1.0196424 ,  0.5600806 ,
#         0.4816082 ,  0.27453795, -1.2056696 , -0.42269334, -0.59054685,
#        -0.67677987, -0.17933069,  0.73068076, -0.9498321 ,  1.9444176 ,
#         0.02687632,  0.52713907, -0.19400337, -0.31508628, -1.3735138 ,
#        -1.0818607 , -0.33657622,  0.17982751,  0.8827995 ,  0.15030423,
#         0.22431774,  0.80091304, -0.03566883,  0.43316683, -0.37539425,
#         0.21121722, -0.11639111,  1.0299757 , -1.002826  ,  0.6939474 ,
#        -0.9105419 , -1.2600976 , -1.6416855 , -0.37046993, -0.19562039]

# other_embedding_of_UnitedStates = np.array(other_embedding_of_UnitedStates)

In [65]:
# np.dot(other_embedding_of_UnitedStates, model.wv[word_to_w2v_generated("United_States")])/(norm(other_embedding_of_UnitedStates)*norm(model.wv[word_to_w2v_generated("United_States")]))

In [66]:
# model.wv[word_to_w2v_generated("United_States")]

In [67]:
model.wv.similarity(word_to_w2v_generated("United_States"), word_to_w2v_generated("isCitizenOf"))

0.07402058

In [68]:
model.wv.most_similar(word_to_w2v_generated("United_States"))

[('54017', 0.8111827969551086),
 ('146871', 0.8006814122200012),
 ('271774', 0.7898718118667603),
 ('188990', 0.780306339263916),
 ('292447', 0.773094117641449),
 ('290840', 0.7530125379562378),
 ('35158', 0.7364202737808228),
 ('5462', 0.7234049439430237),
 ('178828', 0.7059782147407532),
 ('22333', 0.6848352551460266)]

In [69]:
print(key_list[val_list.index('33202')])
print(key_list[val_list.index('121089')])
print(key_list[val_list.index('116524')])
print(key_list[val_list.index('127419')])
print(key_list[val_list.index('117033')])
print(key_list[val_list.index('294331')])
print(key_list[val_list.index('51950')])
print(key_list[val_list.index('63038')])
print(key_list[val_list.index('88756')])
print(key_list[val_list.index('167956')])

Hugo_de_Vries
pl/Konstantin_Żygałow
Karlheinz_Förster
Eduardo_Dato
pl/Michał_Kmieciak
Alsterdorfer_Sporthalle
Stig_Dagerman
Country_Yossi
Seoul_E-Land_FC
Andrew_Cassels


In [70]:
model.wv.most_similar(word_to_w2v_generated("Godavari_Dam"))

[('49331', 0.9982098937034607),
 ('104367', 0.9979304671287537),
 ('17567', 0.9977748990058899),
 ('47474', 0.9977458119392395),
 ('197123', 0.9973745346069336),
 ('42972', 0.997218668460846),
 ('154931', 0.9971106052398682),
 ('138851', 0.9971092343330383),
 ('165414', 0.9970056414604187),
 ('64900', 0.9970017671585083)]

In [71]:
print(key_list[val_list.index('16313')])
print(key_list[val_list.index('189211')])
print(key_list[val_list.index('186779')])
print(key_list[val_list.index('112539')])
print(key_list[val_list.index('208998')])
print(key_list[val_list.index('89406')])
print(key_list[val_list.index('12018')])
print(key_list[val_list.index('22731')])
print(key_list[val_list.index('236631')])
print(key_list[val_list.index('256582')])

Moncton_Stadium
de/Martin_Sommerhoff
Jean-Baptiste_Mills
Vandana_Gupte
Dorothea_Waddingham
Hassan_Tabatabaei_Qomi
Gibson_Burrell
DivX
János_Mihálkovics
Cagnaccio_di_San_Pietro


In [72]:
model.wv.similarity(word_to_w2v_generated('Godavari_Dam'), word_to_w2v_generated('Shirvata_Dam'))

0.9973744

In [73]:
model.wv.most_similar(word_to_w2v_generated("India"))

[('83362', 0.874811053276062),
 ('171541', 0.866265058517456),
 ('264884', 0.45664671063423157),
 ('23878', 0.4491610527038574),
 ('175501', 0.44832557439804077),
 ('125530', 0.4455820620059967),
 ('166336', 0.4454866647720337),
 ('215573', 0.4398549795150757),
 ('182534', 0.437621533870697),
 ('174832', 0.4347749948501587)]

In [74]:
print(key_list[val_list.index('258275')])
print(key_list[val_list.index('216979')])
print(key_list[val_list.index('150019')])
print(key_list[val_list.index('195507')])
print(key_list[val_list.index('115550')])
print(key_list[val_list.index('271126')])
print(key_list[val_list.index('173883')])
print(key_list[val_list.index('302656')])
print(key_list[val_list.index('175975')])
print(key_list[val_list.index('102158')])

Vanuatu_Republican_Party
Henry_Gleditsch
MC_Steppa
Owen_McAleer
pl/Szymon_Jakubowski
Bethany_Campbell
Dempo_S.C.
Mike_Voight
Durduri_District
fr/Jean-Yves_Sénant


In [75]:
model.wv.most_similar(word_to_w2v_generated("PTA_Bank"))

[('30621', 0.9982998371124268),
 ('302365', 0.9971498250961304),
 ('18844', 0.9968996047973633),
 ('242381', 0.9955061674118042),
 ('300928', 0.9952936768531799),
 ('56089', 0.9945706725120544),
 ('106788', 0.9939236640930176),
 ('195867', 0.9924242496490479),
 ('86209', 0.9919241070747375),
 ('304421', 0.9842355251312256)]

In [76]:
print(key_list[val_list.index('67752')])
print(key_list[val_list.index('230549')])
print(key_list[val_list.index('74086')])
print(key_list[val_list.index('276049')])
print(key_list[val_list.index('201431')])
print(key_list[val_list.index('13859')])
print(key_list[val_list.index('91335')])
print(key_list[val_list.index('174332')])
print(key_list[val_list.index('247691')])
print(key_list[val_list.index('66852')])

de/Beat_Tschümperlin
Sagari_Chhabra
Hillar_Teder
Louis_Charles_Auguste_Steinheil
Fathali_Oveisi
Paul_Schneider_(soccer)
de/Dietrich_Arndt
Michael_Teo_Yu_Keng
Don_Nehlen
Cecil_John_Kinross


In [77]:
model.wv.most_similar(word_to_w2v_generated("Air_force_command_building"))

[('117485', 0.997599184513092),
 ('261489', 0.9941193461418152),
 ('218299', 0.9757266044616699),
 ('226487', 0.8765226602554321),
 ('230003', 0.8672513365745544),
 ('19558', 0.8534029126167297),
 ('292130', 0.8404415845870972),
 ('14155', 0.81370609998703),
 ('94169', 0.8076338171958923),
 ('270884', 0.8044722080230713)]

In [78]:
print(key_list[val_list.index('125674')])
print(key_list[val_list.index('152849')])
print(key_list[val_list.index('265359')])
print(key_list[val_list.index('222793')])
print(key_list[val_list.index('134200')])
print(key_list[val_list.index('94175')])
print(key_list[val_list.index('281951')])
print(key_list[val_list.index('241113')])
print(key_list[val_list.index('40657')])
print(key_list[val_list.index('301277')])

Albertine_Caron-Legris
Cavanagh,_Edmonton
Meadowood_Mall
Bai_Jinian
Tarikonda_Venkamamba
Cory_Hightower
Natalia_Kuziutina
Marimar_Vega
James_Edward_Michaels
Thomas_J._Godfrey


## References Used
1) https://datacarpentry.org/python-socialsci/11-joins/index.html#:~:text=The%20concat()%20function%20can,of%20one%20to%20the%20other.&text=The%20merge()%20function%20is%20equivalent%20to%20the%20SQL%20JOIN,inner'%20joins%20are%20all%20possible. \
2) https://sparkbyexamples.com/pyspark/pyspark-replace-column-values/ \
3) https://stackoverflow.com/questions/44626986/merge-multiple-columns-into-one-column-in-pyspark-dataframe-using-python \
4) https://towardsdatascience.com/guide-to-big-data-joins-python-sql-pandas-spark-dask-51b7f4fec810